# Learning feature extraction operator

idea:
learn a convolutional filter which extracts the features map on both target images and source inmages for the purpose that
1.  a small deviation from pose of target just bring samall change on distance of two feature map

2. target pose is the best optimized pose (golab minimum)

Dataset:
- target images from synthetic dataset with airplane pose (position, rotation)

- generate a batch of source images around each target image, with a slowly gradual increased deviation from target images

learning process:
- convolutional filters (one layer or several layers) will be appiled to a pair of images (a target image, and a source image near the target image) -> output two digits (or two vectors?)

- final output is distance between the two oupts,  

- ground truth can be acquired by calculating the discatance between
  souce image pose and target image pose ( how to design the distance of two pose? )

Potential issue:
- the alogrithm could be material-dependent. which mean it maybe doesn't work for real airplane image in real world which have different material, in spite of the same shape .



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://raw.githubusercontent.com/sfu-bigdata/range-driver/public/range_driver/dict_utils.py

In [2]:
!pip install pyrender
!pip install --upgrade redner-gpu
!pip install torchsummary

     |████████████████████████████████| 1.2 MB 7.4 MB/s 
     |████████████████████████████████| 2.2 MB 54.2 MB/s 
     |████████████████████████████████| 890 kB 71.1 MB/s 
     |████████████████████████████████| 640 kB 73.6 MB/s 
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745210 sha256=fc090be1205be24e28a96b60feb29787f29c5b52e8812e49fc5603e4ca4725a1
  Stored in directory: /root/.cache/pip/wheels/c6/83/cb/af51a0c06c33d08537b941bbfc87469e8a3c68d05f77a6a212
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.5
    Uninstalling PyOpenGL-3.1.5:
      Successfully uninstalled PyOpenGL-3.1.5
     |████████████████████████████████| 31.8 MB 1.2 MB/s 


In [5]:
%cd /content/drive/MyDrive/BigDataHub/Differentiable_Render/

/content/drive/MyDrive/BigDataHub/Differentiable_Render


In [8]:
import pyredner
import torch
import redner
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pyrender
import trimesh.transformations as transformations
import os
import h5py
import math
from matplotlib.pyplot import figure
from PIL import Image
from torchsummary import summary
import pandas as pd
import random
import cv2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import PIL.Image as Image
import torchvision.transforms as transforms
import time
# install BDH library:  !wget https://raw.githubusercontent.com/sfu-bigdata/range-driver/public/range_driver/dict_utils.py
from dict_utils import *
from utils_redner_transform import *
from training_model_zoo import *
from utils import *

## Construct custom Dataset

In [ ]:
#each target image correspond to a sequence of source images.
class AnchorDataset_seq(Dataset):
  def __init__(self, dir_h5, dstfile_h5, seq_s_num, transform=None, applySeg=True, greyScale=False):
    # training_s_dir: the directory which stored the sourece images
    # training_t_dir: the directory which stored the target images
    # seq_s_num: number of source images per each sequence

    self.transform = transform
    self.img_paths = []
    self.pose_labels = []
    self.seq_s_num = seq_s_num
  
    #read data from h5 file
    file_p = os.path.join(dir_h5, dstfile_h5)
    self.file_dataset = h5py.File(file_p, 'r')
    print('---', self.file_dataset.keys())
          
  def __len__(self):
      return self.file_dataset['Train_im_t'].shape[0]

  def __getitem__(self, index):
    #get image path
    # egt index of target images
    img_t = self.file_dataset['Train_im_t'][index]
    pose_t = self.file_dataset['Train_p_t'][index]
    if self.transform is not None:
      img_t_input = self.transform(Image.fromarray(img_t.astype('uint8')))

    img_s_seq = torch.tensor([])
    pose_s_seq = np.zeros((self.seq_s_num,7))
    for i in range(1, self.seq_s_num+1):
      img_s = self.file_dataset['Train_im_s'+str(i)][index]
      pose_s = self.file_dataset['Train_p_s'+str(i)][index]  
      if self.transform is not None:
        img_s_input = self.transform(Image.fromarray(img_s.astype('uint8')))

      # wrap source image sequence, add a dimension for number of source img in each sequence:
      # so shape of source images input is (None, seq_s_num, c, h, w)
      img_s_seq = torch.cat((img_s_seq, img_s_input.unsqueeze(0)), 0)
      pose_s_seq[i-1] = pose_s
      
    return img_t_input, img_s_seq, pose_t, pose_s_seq

     

## Framwork
- romove the background to target images ( use segmentation )
- source images are renderred normal image
- architecture of the network:
  - reference from CNN? 
  - output: vecters or digites ?
  - labes: pose distance between source and target
  - loss function: MSE?

## Train model class 

In [9]:
class Train_model():
    def __init__(self, ymal_path):
        # load following object from configuration ymal file
        # 0 config = load_ymal()
        # 1. model architecture
        # 2. loss function
        # 3. dataloader 

        self.conf = yload(open(ymal_path))
        cf_ml = self.conf.ml_model
        self.save_model_name = f"{cf_ml.model_cls_name}_{cf_ml.train_type}"
        self.save_model_path = None

        #  load model
        self.net = None
        self.criterion_seq =None

        #datasert 
        self.train_dataloader = None
        self.val_dataloader = None
        self.test_dataloader = None   

        #optimizer
        self.optimizer = None

        #track training process info
        self.train_loss_profile = []
        self.val_loss_profile =[]

        self.train_loss_terms_profile = []
        self.val_loss_terms_profile = []

        self.train_model_output = []
        self.val_model_output = []
      

    def load_model(self):
        cf_ml_model =self.conf.ml_model
        model_cls_name = cf_ml_model.model_cls_name
        
        #convert str to class and initialize net obj
        self.net = str_to_class('model_cls_name')()
        
        #visualize model summary
        # input1_shape = (3,300,400)
        # input2_shape = (3,300,400)
        # visualize_model_summary(input1_shape, input2_shape)

        #loss function
        #criterion = nn.MSELoss()
        cf_loss_f = cf_ml_model.loss_f
        self.criterion_seq = loss_seq_fn(cf_loss_f)        
    
    def set_dataloader(self):
        #initialize the dataset
        # Load the the dataset from raw image folders
        cf_dl = self.conf.dataloader
        cf_tfr = cf_dl.transformer

        if cf_tfr.Without_resize:
          print("Inputs without resizing")
          custom_trans = transforms.Compose([transforms.ToTensor(), 
                                            transforms.Normalize(cf_tfr.norm_mean, cf_tfr.norm_std)])
        else:
          custom_trans = transforms.Compose([transforms.Resize(cf_tfr.nput_imgsize),
                                             transforms.ToTensor(),
                                            transforms.Normalize(cf_tfr.norm_mean, cf_tfr.norm_std)])


        anchor_dataset = AnchorDataset_seq(cf_dl.dir_h5, cf_dl.train_h5_name, cf_dl.n_sample, transform=custom_trans)
        val_dataset = AnchorDataset_seq(dir_h5, cf_dl.val_h5_name, cf_dl.n_sample, transform=custom_trans )
        test_dataset = AnchorDataset_seq(dir_h5, cf_dl.test_h5_name, cf_dl.n_sample, transform=custom_trans)

        #datasert 
        self.train_dataloader = DataLoader(anchor_dataset, batch_size = 4, shuffle= False)
        self.val_dataloader = DataLoader(val_dataset, batch_size = 4, shuffle= False)
        self.test_dataloader = DataLoader(test_dataset, batch_size = 1)      

        
    def model_train(self, epochs):
        conf_train = self.conf.train
        conf_opt = conf_train.optimizer
        
        # continue train or not
        if conf_train.continue_train:
          try:
            # load the checkpoint dict
            checkpoint_dict_name = f"{self.save_model_name}_last_checkpoint.pt"
            net.load_state_dict(torch.load(conf_train.save_dir+ checkpoint_dict_name))
            print("model loaded successfully", checkpoint_dict_name)
          except:
            print("continuous training, but no model exists")   

        # set optimizer
        self.optimizer = torch.optim.Adam(net.parameters(), lr=conf_opt.lr, weight_decay = conf_opt.weight_decay)
        #optimizer = torch.optim.Adagrad(net.parameters())
        #optimizer = torch.optim.Adadelta(net.parameters())       

        # start
        self.net.cuda()

        start = time.time()
        for epoch in range(epochs):
          r_count = 0
          total_loss = 0
          val_total_loss = 0
          total_train_loss_terms = np.zeros((1,3))
          
     #========================================================
          #------training part ---------------------------------------------------
          net.train()

          for row_id, data in enumerate(self.train_dataloader, 0):
            img_t, img_s_seq, pose_t, pose_s_seq = data            
            batch_size = img_s_seq.size()[0]
            seq_s_num = img_s_seq.size()[1]
            
            # structure the input data 
            # breakdown (batach_size, seq_s_num,c,h,w) -> (seq_s_num,c,h,w), which is feed into model
            for i in range(batch_size): 
              r_count += 1
        
              img_s_input = img_s_seq[i] 
              pose_s_input = pose_s_seq[i]

              #expand dimension of the target image
              tg_im  = img_t[i]
              tg_pose = pose_t[i]

              tg_im = tg_im.repeat(seq_s_num, 1,1,1)
              tg_pose = tg_pose.repeat(seq_s_num,1)

              # move to GPU
              tg_im = tg_im.cuda()
              img_s_input = img_s_input.cuda()
              tg_pose = tg_pose.cuda()
              # poses don't use in compute loss function, tentatively 
              pose_s_input = pose_s_input.cuda()
              
              # set zero gradient
              self.optimizer.zero_grad()
              # forward output
              output = net(tg_im, img_s_input)
              # calculate the loss
              #loss = criterion(out_t, out_s, dif_pos)
              loss, train_loss_terms = self.criterion_seq(output)

              #===============record training info--------------------
              if epoch%5 == 0:  
                print(f"==index: {row_id*4+i}==train output ====", [epoch, row_id, i] + torch.flatten(output).cpu().data.tolist())

              #save the output distance of sweep 
              self.train_model_output.append([epoch, row_id, i] + torch.flatten(output).cpu().data.tolist())

              #record the loss
              total_loss += loss.item() 
              total_train_loss_terms += np.array(train_loss_terms)

              # loss terms for each sample
              train_index_info = np.array([epoch, row_id, i])
              tran_loss_terms_instance = np.concatenate((train_index_info, np.array(train_loss_terms)), axis=None)
              self.train_loss_terms_profile.append(tran_loss_terms_instance)


              #----------back propagation------------
              loss.backward()
              self.optimizer.step()
          
          t= time.time()-start
          #save training loss and print
          self.train_loss_profile.append(total_loss/r_count)
          
          print("========   epoch: {} ========".format(epoch+1))  
          print("[training] Time:{}  dataset siz:{}".format(t, r_count))
          print("   [training] Loss: {}".format(total_loss/r_count))
          print("   [traning] Loss terms:{} ".format(total_train_loss_terms/r_count))
        
          #save checkpoints of last epoch
          if epoch == epochs -1:
              # save weight dict 
              last_cp_path = conf_train.save_dir + f"{self.save_model_name}_last_checkpoint.pt"
              torch.save(net.state_dict(), last_cp_path) 

    ## ===========================================================+++++++++++++++++++++++++
          ## -----validation part-----------------------
          r_val_count = 0
          total_val_loss_term = np.zeros((1,3))
          net.eval()
          for row_id, data in enumerate(self.val_dataloader, 0):
            img_t, img_s_seq, pose_t, pose_s_seq = data 
            
            batch_size = img_s_seq.size()[0]
            seq_s_num = img_s_seq.size()[1]
            
            # breakdown (batach_size, seq_s_num,c,h,w) -> (seq_s_num,c,h,w), which is feed into model
            for i in range(batch_size): 
              r_val_count += 1

              img_s_input = img_s_seq[i] 
              pose_s_input = pose_s_seq[i]

              #expand dimension of the target image
              tg_im  = img_t[i]
              tg_pose = pose_t[i]

              tg_im = tg_im.repeat(seq_s_num, 1,1,1)
              tg_pose = tg_pose.repeat(seq_s_num,1)

              # move to GPU
              tg_im = tg_im.cuda()
              img_s_input = img_s_input.cuda()
              tg_pose = tg_pose.cuda()
              pose_s_input = pose_s_input.cuda()

              output = net(tg_im, img_s_input)

            #---------------record validation info---------------------------------------------------------------
              if epoch%5 == 0:
                print("====== val output =====",[epoch, row_id, i] + torch.flatten(output).cpu().data.tolist())
              #out_t, out_s = net(img_t, img_s)
              
              #save lase epoch output
              self.val_model_output.append([epoch, row_id, i] + torch.flatten(output).cpu().data.tolist())

              # calculate the loss
              #loss = criterion(out_t, out_s, dif_pos)
              val_loss, val_loss_term = self.criterion_seq(output)
              #t1, t2, t3 = compute_loss_comp(output)
              val_total_loss += val_loss.item() 
              
              # record loss terms for validation data
              total_val_loss_term += np.array(val_loss_term)
              # loss terms for each sample
              val_index_info = np.array([epoch, row_id, i])
              val_loss_terms_instance = np.concatenate((val_index_info, np.array(val_loss_term)), axis=None)
              self.val_loss_terms_profile.append(val_loss_terms_instance)      
          
          t= time.time()-start
          #print 

          #save val loss and print
          self.val_loss_profile.append(val_total_loss/r_val_count)
          self.val_loss_terms_profile.append(total_val_loss_term/r_val_count)

          print("[val] Time:{}, size: {} ".format(t,r_val_count))
          print("   [val] Loss: {}".format(val_total_loss/r_val_count))
          print("   [val] Loss terms:{} ".format(total_val_loss_term/r_val_count))

        
        ##------------------------------
        ## save the entire model
        self.save_model_path = conf_train.save_dir + f"{self.save_model_name}.pt"
        torch.save(net, self.save_model_path)  
        print("save succesfully")  
    

    def evaluate_on_testdata(self, mode=0):
        # load saved network or use  self.net
        # mode 0: use current network   1. load from disk
        if mode ==0:
          net = self.net
        else:
          try:
            net = torch.load(self.save_model_path)
            print("model loaded successfully")
          except Exception as e:
            print("Fails to load model")
            print(e)
        
        #send to gpu
        net.eval()  
        net.cuda()
        start = time.time()
        distance_profiles = []
        losses = []
        for row_id, data in enumerate(self.test_dataloader, 0):
          img_t, img_s_seq, pose_t, pose_s_seq = data 
          
          batch_size = img_s_seq.size()[0]
          seq_s_num = img_s_seq.size()[1]
            
          # breakdown (batach_size, seq_s_num,c,h,w) -> (seq_s_num,c,h,w), which is feed into model
          for i in range(batch_size): 

            img_s_input = img_s_seq[i] 
            pose_s_input = pose_s_seq[i]

            #expand dimension of the target image
            tg_im  = img_t[i]
            tg_pose = pose_t[i]

            tg_im = tg_im.repeat(seq_s_num, 1,1,1)
            tg_pose = tg_pose.repeat(seq_s_num,1)

            # move to GPU
            tg_im = tg_im.cuda()
            img_s_input = img_s_input.cuda()
            tg_pose = tg_pose.cuda()
            pose_s_input = pose_s_input.cuda()
            output = net(tg_im, img_s_input)

            # check if the sequence of distances is monotonical increased 
            dd = output[1:] - output[:-1]
            t1,t2,t3 = compute_loss_comp(output)
            loss =  t1+t2+t3

            print('dd: ', dd.T.data.cpu()[0].tolist())
            print("loss terms : ", t1, t2, t3  )
            print("loss:", loss.item())

            #print(dd.flatten().data.cpu().tolist())
            distance_profiles.append(dd.flatten().data.cpu().tolist())
            losses.extend(loss.cpu().tolist())
          
          t = time.time()-start
          ## validation 
          ## size t : s = 10:300 
        
        # plot loss and output distance anaysis on test data
        self.plot_dd_by_top_loss(losses, distance_profiles)


    def plot_dd_by_top_loss(self, losses, distance_profiles, num_top = 20):

        # retrieve metadata from metadata_test_sequence.txt
        test_metadata_p = self.conf.project_base_dir + "dataset/metadata_test_sequence.txt"
        df_st = pd.read_csv(test_metadata_p, header=None)
        df_st[21] = None
        df_st.loc[:,12:21] = df_st.loc[:,11:20].values
        df_st.loc[:,10:11] = df_st.loc[:,10].str.split(expand=True).values

        df = pd.DataFrame({'loss':losses, 'd_distances':distance_profiles})
        df.sort_values(by='loss')
        num_top = 30
        for idx, r in df.sort_values(by='loss', ascending=True).head(num_top).iterrows():
          # get the information of sequence from the file names:

          perturbation_type = df_st.iloc[idx,1][:-7]
          figure(figsize=(8,6))
          #plt.plot(np.cumsum(r['d_distances']))
          plt.plot(r['d_distances'])
          plt.plot([0,9],[0,0])
          plt.title(f"Example: {idx} Loss: {r['loss']:.2f}  sweep_type:{perturbation_type}")
          plt.ylabel('dd_k')
          plt.xlabel('k')
          plt.show()




    def analysis_after_training(self):
        output_dir = self.conf.train.output_result_dir

        train_model_output_pd = pd.DataFrame(self.train_model_output, columns=("epoch","batch_id","index","d0","d1","d2","d3","d4","d5","d6","d7","d8","d9"))
        val_model_output_pd = pd.DataFrame(self.val_model_output, columns=("epoch","batch_id","index","d0","d1","d2","d3","d4","d5","d6","d7","d8","d9"))

        train_loss_detail_pd = pd.DataFrame(self.train_loss_terms_profile, columns=("epoch","batch_id","index","t1","t2","t3"))
        val_loss_detail_pd = pd.DataFrame(self.val_loss_terms_profile, columns=("epoch","batch_id","index","t1","t2","t3"))

        train_last_epoch = train_model_output_pd.iloc[-725:,:]
        val_last_epoch = val_model_output_pd.iloc[-725:,:]
        
       #--------------------------------------------------------------------------------------------
        # filter the increasing sweep in last epoch
        t_selection = train_last_epoch[(train_last_epoch['d1']>train_last_epoch['d0']) &
                        (train_last_epoch['d2']>train_last_epoch['d1']) &
                        (train_last_epoch['d3']>train_last_epoch['d2']) &
                        (train_last_epoch['d4']>train_last_epoch['d3']) &
                        (train_last_epoch['d5']>train_last_epoch['d4']) &
                        (train_last_epoch['d6']>train_last_epoch['d5']) &
                        (train_last_epoch['d7']>train_last_epoch['d6']) &
                        (train_last_epoch['d8']>train_last_epoch['d7']) &
                        (train_last_epoch['d9']>train_last_epoch['d8']) ]

        v_selectoion = val_last_epoch[(val_last_epoch['d1']>val_last_epoch['d0']) &
                        (val_last_epoch['d2']>val_last_epoch['d1']) &
                        (val_last_epoch['d3']>val_last_epoch['d2']) &
                        (val_last_epoch['d4']>val_last_epoch['d3']) &
                        (val_last_epoch['d5']>val_last_epoch['d4']) &
                        (val_last_epoch['d6']>val_last_epoch['d5']) &
                        (val_last_epoch['d7']>val_last_epoch['d6']) &
                        (val_last_epoch['d8']>val_last_epoch['d7']) &
                        (val_last_epoch['d9']>val_last_epoch['d8']) ]
       
        print(f"{t_selection.shape[0]} records in traing meet the requirement")
        print(t_selection)

        print(f"{v_selectoion.shape[0]} records in validation meet the requirement")
        print(v_selectoion)

        # group by epochs
        total_loss_totoal_pd = train_loss_detail_pd.groupby("epoch").mean()
        val_loss_totoal_pd = val_loss_detail_pd.groupby("epoch").mean()
        
    #============================================================================
        train_loss_profile = np.sum(total_loss_totoal_pd.to_numpy()[:,2:], axis = 1)
        val_loss_profile = np.sum(val_loss_totoal_pd.to_numpy()[:,2:], axis = 1)

        figure(figsize=(8,6))
        plt.title("Training and validation loss")
        plt.plot(train_loss_profile)
        plt.plot(val_loss_profile)
        plt.ylabel('loss')
        plt.xlabel('epochs')
        plt.show()

     #--------------------------------------------------------------------------
        # save loss detail to csv files
        train_detail_name = self.save_model_name + "_t_loss_detail.csv"
        val_detail_name = self.save_model_name +"v_loss_detail.csv"
        train_loss_detail_pd.to_csv(output_dir + train_detail_name, index=False)
        val_loss_detail_pd.to_csv(output_dir + val_detail_name, index=False)


        m_output_train_name  = self.save_model_name + "_t_output.csv.csv"
        m_output_val_name = self.save_model_name + "_v_output.csv.csv"
        train_model_output_pd.to_csv(output_dir + m_output_train_name, index=False)
        val_model_output_pd.to_csv(output_dir + m_output_val_name, index=False)


    def sample_loss_term(self, batch_id=1, index_inbatch=2, mode="training"):
        #loss terms for certain samples throught the training
        train_loss_detail_pd = pd.DataFrame(self.train_loss_terms_profile, columns=("epoch","batch_id","index","t1","t2","t3"))
        val_loss_detail_pd = pd.DataFrame(self.val_loss_terms_profile, columns=("epoch","batch_id","index","t1","t2","t3"))

        if mode == "training":
           sample_loss = train_loss_detail_pd[(train_loss_detail_pd['batch_id'] == batch_id) & (train_loss_detail_pd['index'] == index_inbatch) ]
        elif mode == "val":
           sample_loss = val_loss_detail_pd[(val_loss_detail_pd['batch_id'] == batch_id) & (val_loss_detail_pd['index'] == index_inbatch) ]
        else:
           print("Fail to load mode, [training, val]")
           return

        sample_t1_profle = sample_loss.to_numpy()[:,3]
        sample_t2_profle = sample_loss.to_numpy()[:,4]
        sample_t3_profle = sample_loss.to_numpy()[:,5]
        sample_loss_profole = np.sum(sample_loss.to_numpy()[:,2:], axis = 1)

        figure(figsize=(8,6))
        #plt.plot(np.cumsum(r['d_distances']))
        plt.plot(sample_loss_profole)
        plt.plot(sample_t1_profle)
        plt.plot(sample_t2_profle)
        plt.plot(sample_t3_profle)
        plt.legend(['total loss', 't1', 't2','t3'])
        plt.title(f"Example: batch id: {batch_id}, index in batch {index_inbatch}")
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.show()

## Train network with image sequences (Architecture 2)

In [ ]:
base_dir = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/conf/'

ymal_path = "training_config.yml"

model_training = Train_model( base_dir + ymal_path )

## training

In [ ]:
epochs = 40
model_training.model_train(epochs):

## evaluate on test dataset

In [ ]:
model_training.evaluate_on_testdata()

## Analysis after training
- review qualified training and validate data
- save train ouput and loss detail on disk 

In [ ]:
model_training.analysis_after_training()

- Observe loss term change for a perticular sample during training process

In [ ]:
b_id = 1
idx_inb = 2
mode = "training"  # "training " or "val"
model_training.sample_loss_term(batch_id=b_id, index_inbatch=idx_inb, mode=mode)